In [ ]:
!pip install langchain openai pinecone-client langchain-community sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 3.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import pinecone
from pinecone import Pinecone, ServerlessSpec

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
# 1. 모델 로드 (Sentence-BERT)
from sentence_transformers import SentenceTransformer
from nltk.tokenize import sent_tokenize
import numpy as np

# multi-qa-mpnet-base-dot-v1 모델 로드
#model = SentenceTransformer('sentence-transformers/multi-qa-mpnet-base-dot-v1')
model = SentenceTransformer('sentence-transformers/paraphrase-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
model.get_sentence_embedding_dimension()

384

In [ ]:
# 2. 파인콘 초기화
from pinecone import Pinecone, ServerlessSpec

pinecone = Pinecone(api_key="your-api-key")
index = pinecone.Index("trip-index")

# Tour

In [ ]:
# 3. 검색 함수 정의

# 여행 스타일 및 동행인 기반
def search_places_style(city, companions, travel_style):
    query = f"Best places in {city} for {companions} with focus on {travel_style}."
    query_embedding = model.encode(query).tolist()
    namespace = f"{city}_tour"
    results_style= index.query(vector=query_embedding, top_k=30, namespace=namespace, include_metadata=True)
    return results_style

# 기본 유명 관광지
def search_places(city):
    query = f"The most famous tour places in {city}"
    query_embedding = model.encode(query).tolist()
    namespace = f"{city}_tour"
    results_best= index.query(vector=query_embedding, top_k=50, namespace=namespace, include_metadata=True)
    return results_best

In [ ]:
# 쿼리 벡터 생성 - 파리 / 여행일정
city = 'paris'
companions = 'parents'
travel_style = 'shopping', 'dining', 'nightlife'
results_style = search_places_style(city, companions, travel_style)
results_best = search_places(city)

In [ ]:
print(len(results_style['matches']))
print(len(results_best['matches']))

30
50


In [ ]:
import pandas as pd

# 검색 결과를 CSV 파일로 저장하는 함수
def save_results_to_csv(results_style, results_best, file_name="search_results.csv"):
    # 결과에서 필요한 데이터를 추출

    # results_style
    data1 = []
    for result in results_style['matches']:
        row = {
            'ID': result['id'],
            'Score': result['score']
        }
        # 메타데이터가 포함되어 있을 경우 추가
        row.update(result.get('metadata', {}))
        data1.append(row)

    # results_best
    data2 = []
    for result in results_best['matches']:
        row = {
            'ID': result['id'],
            'Score': result['score']
        }
        # 메타데이터가 포함되어 있을 경우 추가
        row.update(result.get('metadata', {}))
        data2.append(row)

    # pandas DataFrame으로 변환
    df1 = pd.DataFrame(data1)
    df2 = pd.DataFrame(data2)
    df = pd.concat([df1, df2], ignore_index=True)
    df = df.drop_duplicates(subset=['ID'])     # 중복해서 나온 장소 제거

    # CSV 파일로 저장
    df.to_csv(file_name, index=False)
    print(f"Results saved to {file_name}")

# 검색 결과를 CSV 파일로 저장
save_results_to_csv(results_style, results_best, file_name="paris_search_results.csv")

Results saved to paris_search_results.csv


# Lodging

In [ ]:
# 3. 검색 함수 정의

# 선호 숙소 형태 및 동행인 기반
def search_places(city, companions, lodging_style):
    query = f"Best accommdations in {city} for {companions} with focus on {lodging_style}."
    query_embedding = model.encode(query).tolist()
    namespace = f"{city}_lodging"
    results_style = index.query(vector=query_embedding, top_k=20, namespace=namespace, include_metadata=True)
    return results_style

In [ ]:
# 쿼리 벡터 생성 - 파리 / 숙소
city = 'paris'
companions = 'parents'                      # alone
lodging_style = 'luxury', 'swimming pool'    # cheap, breakfast
results_lodging = search_places(city, companions, lodging_style)

In [ ]:
results_lodging